In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
from models import lstm
import torch

In [2]:
# FILES TO CHANGE (This is where your sequence_nlp_harvey.zip etc is) 

NPY_INPUT_DIR = '/Users/ianmagnusson/IITUDND/data/extracted_features/combined_NLP/harvey/'

In [3]:
# load data from files

X_seq_tfidf_train = np.load(NPY_INPUT_DIR + 'X_seq_tfidf_train.npz')
X_seq_tfidf_test = np.load(NPY_INPUT_DIR + 'X_seq_tfidf_test.npz')


X_seq_glove_train = np.load(NPY_INPUT_DIR + 'X_seq_glove_train.npz')
X_seq_glove_test = np.load(NPY_INPUT_DIR + 'X_seq_glove_test.npz')

y_train = torch.Tensor(np.load(NPY_INPUT_DIR + 'y_train.npy'))
y_test = torch.Tensor(np.load(NPY_INPUT_DIR + 'y_test.npy'))

In [4]:
# format npz objects as lists of arrays again
X_tfidf_train = [ torch.Tensor(X_seq_tfidf_train[file]) for file in X_seq_tfidf_train.files]
X_tfidf_test = [ torch.Tensor(X_seq_tfidf_test[file]) for file in X_seq_tfidf_test.files]

X_glove_train = [ torch.Tensor(X_seq_glove_train[file]) for file in X_seq_glove_train.files]
X_glove_test = [ torch.Tensor(X_seq_glove_test[file]) for file in X_seq_glove_test.files]

In [5]:
# build lists with features concatenated

X_train = []
for i in range(len(y_train)):
    file = X_seq_tfidf_train.files[i]
    X_train.append(torch.Tensor(np.concatenate((X_seq_tfidf_train[file], X_seq_glove_train[file]), axis=1)))
    
X_test = []
for i in range(len(y_test)):
    file = X_seq_tfidf_test.files[i]
    X_test.append(torch.Tensor(np.concatenate((X_seq_tfidf_test[file], X_seq_glove_test[file]), axis=1)))


In [6]:
# build and train model

model = lstm.LSTM(input_dim=400, hidden_dim=400)

#model.learn(X_train, y_train)
#torch.save(model.state_dict(), 'lstm.pt')

model.load_state_dict(torch.load('lstm.pt'))
model.eval()


epoch: 0 learning rate: [0.01]
[1,   200] loss: 0.495
[1,   400] loss: 0.533
[1,   600] loss: 0.511
[1,   800] loss: 0.554
[1,  1000] loss: 0.442
[1,  1200] loss: 0.484
[1,  1400] loss: 0.441
[1,  1600] loss: 0.458
[1,  1800] loss: 0.471
[1,  2000] loss: 0.395
[1,  2200] loss: 0.463
[1,  2400] loss: 0.436
[1,  2600] loss: 0.473
[1,  2800] loss: 0.308
[1,  3000] loss: 0.413
[1,  3200] loss: 0.397
epoch: 1 learning rate: [0.005]
[2,   200] loss: 0.436
[2,   400] loss: 0.396
[2,   600] loss: 0.287
[2,   800] loss: 0.417
[2,  1000] loss: 0.416
[2,  1200] loss: 0.351
[2,  1400] loss: 0.421
[2,  1600] loss: 0.417
[2,  1800] loss: 0.337
[2,  2000] loss: 0.292
[2,  2200] loss: 0.394
[2,  2400] loss: 0.387
[2,  2600] loss: 0.336
[2,  2800] loss: 0.368
[2,  3000] loss: 0.383
[2,  3200] loss: 0.397


In [11]:
# evaluate
model.get_accuracy(X_test, y_test)

model.get_accuracy_graph(X_test, y_test)


0.8375

In [ ]:
"""
print('here')
y_bootstrapped = []
       
with torch.no_grad():
    for i, X_i in enumerate(X_train):
        if i % 200 == 199:
            print(i)
        predictions = torch.round(model(X_i)).reshape(-1)
        y_bootstrapped.append(predictions)
            
            '''
        y_bootstrapped.append(np.zeros(len(X_i), dtype=np.int_))
        for j in range(len(X_i)):
            outputs = model(X_i[:j + 1])
            predictions = torch.round(outputs).reshape(-1).item()
            y_bootstrapped[i][j] = int(predictions)
            '''
"""